## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

# Весь код, который нужно разобрать, будем дебажить

In [2]:
x=np.array([[3,2],[4,4],[2,4]])

In [3]:
y=np.array([[1], [2], [2]])

In [4]:
sorted_idx = x.argsort()
sorted_idx

array([[1, 0],
       [0, 1],
       [0, 1]], dtype=int64)

сортируем внутри строк

In [5]:
sorted_x, sorted_y = x[sorted_idx], y[sorted_idx]
sorted_x, sorted_y 

(array([[[4, 4],
         [3, 2]],
 
        [[3, 2],
         [4, 4]],
 
        [[3, 2],
         [4, 4]]]), array([[[2],
         [1]],
 
        [[1],
         [2]],
 
        [[1],
         [2]]]))

пока непонятно что, скорее всего передаем ОДИН признак. Попробуем еще раз

In [6]:
x=np.array([7, 6, 5,4,3,2,1,0])
y=np.array([1, 1, 1,1,0,0,2,2])

In [7]:
sorted_idx = x.argsort()
sorted_idx

array([7, 6, 5, 4, 3, 2, 1, 0], dtype=int64)

In [8]:
sorted_x, sorted_y = x[sorted_idx], y[sorted_idx]
sorted_x, sorted_y 

(array([0, 1, 2, 3, 4, 5, 6, 7]), array([2, 2, 0, 0, 1, 1, 1, 1]))

Отсортировали значения по возрастанию признака

In [9]:
class_number = np.unique(y).shape[0]
class_number

3

In [10]:
np.unique(y)

array([0, 1, 2])

Получили количество уникальный классов

In [11]:
splitted_sorted_y = sorted_y
splitted_sorted_y

array([2, 2, 0, 0, 1, 1, 1, 1])

удалил лишнее обрезание, так как это должно выполняться в другом методе

In [12]:
splitted_sorted_y = sorted_y
r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (1)

r_border_ids

array([2, 4], dtype=int64)

получили список, где будем делить (правые границы изменения классов)

Если класс по размеру не разделим, то возвращаем float('+inf'), None

In [13]:
one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
one_hot_code

array([[0., 0., 0.],
       [0., 0., 0.]])

Кодируем классы - количество строк - количество возможных делений, количество колоннок - количество классов

In [14]:
eq_el_count = r_border_ids - np.append([0], r_border_ids[:-1])
eq_el_count

array([2, 2], dtype=int64)

количество одинаковых элементов (после отсечения)

In [15]:
np.arange(r_border_ids.shape[0])

array([0, 1])

получаем индексы для всех строк

In [16]:
sorted_y[r_border_ids - 1]

array([2, 0])

получаем значения левых границ

классы должны быть закодированы, начиная с 0

In [17]:
one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
one_hot_code

array([[0., 0., 1.],
       [1., 0., 0.]])

In [18]:
eq_el_count.reshape(-1, 1)

array([[2],
       [2]], dtype=int64)

In [19]:
class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
class_increments        

array([[0., 0., 2.],
       [2., 0., 0.]])

In [20]:
np.bincount(sorted_y[:], minlength=class_number)

array([2, 4, 2], dtype=int64)

In [21]:
class_increments[0] = class_increments[0] 
class_increments

array([[0., 0., 2.],
       [2., 0., 0.]])

Получили количество классов КРОМЕ находящиегося справа на ВСЕЙ выборке

In [22]:
l_class_count = np.cumsum(class_increments, axis=0)        
l_class_count

array([[0., 0., 2.],
       [2., 0., 2.]])

наконец-то получили количество классов слева при разделении

In [23]:
r_class_count = np.bincount(y) - l_class_count
r_class_count

array([[2., 4., 0.],
       [0., 4., 0.]])

получили количество классов справа

In [24]:
l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
l_sizes

array([[2],
       [4]], dtype=int64)

сумма классов слева

In [25]:
r_sizes = sorted_y.shape[0] - l_sizes
r_sizes

array([[6],
       [4]], dtype=int64)

сумма классов справа

С кодом разобрались. Теперь тестируем критерии информативности.
Так как берется левая и правая часть и берется минимум, то просто берем сумму левой и правой части по информативности

In [26]:
    def gini(l_c, l_s, r_c, r_s):
        #l_class_count, l_sizes, r_class_count, r_sizes
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        sum_l = np.sum(np.power(l_c, 2), axis=1)
        c_l = np.power(l_s.reshape([1, l_s.shape[0]*l_s.shape[1]])[0], 2)
        
        sum_r = np.sum(np.power(r_c, 2), axis=1)
        c_r = np.power(r_s.reshape([1, r_s.shape[0]*r_s.shape[1]])[0], 2)
        return 2 - sum_l/c_l - sum_r/c_r

In [27]:
gini(l_class_count, l_sizes, r_class_count, r_sizes)

array([0.44444444, 0.5       ])

In [28]:
l_class_count, l_sizes, r_class_count, r_sizes

(array([[0., 0., 2.],
        [2., 0., 2.]]), array([[2],
        [4]], dtype=int64), array([[2., 4., 0.],
        [0., 4., 0.]]), array([[6],
        [4]], dtype=int64))

Проверим

In [29]:
1-4/4+1-4/36-16/36, 1-8/16+1-16/16

(0.4444444444444444, 0.5)

Совпало. Идем дальше

In [30]:
l_class_count/l_sizes

array([[0. , 0. , 1. ],
       [0.5, 0. , 0.5]])

In [31]:
def entropy(l_c, l_s, r_c, r_s):
    p_l = l_c/l_s
    p_r = r_c/r_s
    p_ll = np.log2(p_l, where = p_l!=0)
    p_rl = np.log2(p_r,  where = p_r!=0)
    return - np.sum(p_l*p_ll, axis=1) - np.sum(p_r*p_rl, axis=1) 

In [32]:
entropy(l_class_count, l_sizes, r_class_count, r_sizes)

array([0.91829583, 1.        ])

In [33]:
def misclass(l_c, l_s, r_c, r_s):
    p_l = l_c/l_s
    p_r = r_c/r_s
    m_l = np.max(p_l, axis=1)
    m_r = np.max(p_r, axis=1)
    return 2 - m_l - m_r

In [34]:
misclass(l_class_count, l_sizes, r_class_count, r_sizes)

array([0.33333333, 0.5       ])

In [35]:
import math

Теперь определения индексов feature

In [36]:
def get_feature_ids_sqrt(n_feature):
    feature_ids = np.arange(n_feature)
    np.random.shuffle(feature_ids)
    return feature_ids[:int(n_feature**0.5)]
        
def get_feature_ids_log2(n_feature):
    feature_ids = np.arange(n_feature)
    np.random.shuffle(feature_ids)
    return feature_ids[:int(math.log2(n_feature))]

def get_feature_ids_N(n_feature):
    return np.arange(n_feature)

In [37]:
get_feature_ids_sqrt(4)

array([2, 1])

In [38]:
get_feature_ids_log2(8)

array([7, 5, 0])

In [39]:
get_feature_ids_N(3)

array([0, 1, 2])

все кажется нормально

Пусть у нас есть r_border_id и y

In [40]:
r_border_ids = np.array([1,3,4,5,6])
splitted_sorted_y = np.array([0,1,1,2,1,1,2])

Следующая строчка нужна, так как вычисляем длину для y

In [41]:
eq_el_count = r_border_ids - np.append([0], r_border_ids[:-1])
eq_el_count

array([1, 2, 1, 1, 1])

In [42]:
one_hot_code = np.zeros((r_border_ids.shape[0], class_number))


In [43]:
one_hot_code

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [44]:
one_hot_code[np.arange(r_border_ids.shape[0]), splitted_sorted_y[r_border_ids - 1]] = eq_el_count


In [45]:
one_hot_code

array([[1., 0., 0.],
       [0., 2., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [84]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    class Node:
        def __init__(self, type_node = None, feature_id = None, threshold = None, clazz = None, proba = None):
            self.type_node = type_node
            self.feature_id = feature_id,
            self.threshold = threshold
            self.proba = proba
            
            self.__left = None
            self.__right = None
        
        def createLeft(self):
            self.__left = MyDecisionTreeClassifier.Node()
            return self.__left
        
        def createRight(self):
            self.__right = MyDecisionTreeClassifier.Node()
            return self.__right
        
        def getLeft(self):
            return self.__left
        
        def getRight(self):
            return self.__right
        
        def __str__(self):
            return "[" + ",".join( [ str(element) for element in [self.type_node, self.feature_id, self.threshold, self.proba, self.__left, self.__right] ] ) + "]"

    
    def __init__(self, min_samples_split=2, max_depth=None, 
                 sufficient_share=1.0, criterion='gini', max_features=None):
        #комментарии для себя:
        #ожидается в dict:
        #ключ - id, как при хранении дерева в массиве, так:
        # 0 - слева 1, справа 2
        # 1 - слева 2*1+1=3, справа 2*1+2=4
        # 2 - слева 2*2+1=5, справа 2*2+2 = 6
        # В самом объекте ожидается массив из 3 элементов, 
        # первый - код LEAF_TYPE, промежуточные NON_LEAF_TYPE, конечные - LEAF_TYPE
        # для NON_LEAF_TYPE - 2 и 3 - feature_id, threshold
        # для LEAF_TYPE - класс и вероятность
        self.tree = MyDecisionTreeClassifier.Node()
        
        
        # минимальное количество элементов в листе для разделения
        self.min_samples_split = min_samples_split
        #максимальная глубина
        self.max_depth = max_depth
        
        if self.max_depth==None:
            self.max_depth = 20
        #доля преобладающего класса
        self.sufficient_share = sufficient_share
        self.num_class = -1
        #функция критерии
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print ('invalid criterion name')
            raise

        #количество используемых призаков
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print ('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        #l_class_count, l_sizes, r_class_count, r_sizes
        #См. выше в отладке
        return gini(l_c, l_s, r_c, r_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        #См. выше в отладке
        return entropy(l_c, l_s, r_c, r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        #См. выше в отладке
        return misclass(l_c, l_s, r_c, r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        #См. выше в отладке
        return get_feature_ids_sqrt(n_feature)
    
    def __get_feature_ids_log2(self, n_feature):
        #См. выше в отладке
        return get_feature_ids_log2(n_feature)

    def __get_feature_ids_N(self, n_feature):
        #См. выше в отладке
        return get_feature_ids_N(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
                
# см. выше
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        #  см. выше
        splitted_sorted_y = sorted_y
        r_border_ids = np.where((splitted_sorted_y[:-1] != splitted_sorted_y[1:]) & (sorted_x[:-1] != sorted_x[1:]) )[0] + (1)
        #здесь добавим код, который говорит, что и x должны быть разные
        if len(r_border_ids) == 0:
            return None, None
        
        #  см. выше
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), splitted_sorted_y[r_border_ids - 1]] = r_border_ids - np.append([0], r_border_ids[:-1])
        
        #  см. выше
        l_class_count = np.cumsum(one_hot_code, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        
        #как-то считаем критерий информативности, при этом возвращаем массив [1,2,34]
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        #находим минимальное значение
        idx = np.argmin(gs)
    
        # значение границы
        left_el_id = l_sizes[idx][0]
        #возвращаем минимальное значение информативности и значение границы
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
    
    def __fit_node(self, x_m, y_m, node, depth_m):
       
        lst = [[x_m, y_m, node, depth_m]]
        
        while len(lst)>0:
            x, y, node, depth = lst.pop()
            #придется делать через рекурсию, так как такой интерфейс, хотя для python не очень хорошо
            #сначала определим что мы НЕ в LEAF:
            type_leaf = self.NON_LEAF_TYPE
            if self.max_depth is not None and depth >= self.max_depth:
                type_leaf = self.LEAF_TYPE

            share = np.max(np.unique(y, return_counts=True)[1])/y.shape[0]
            if share >= self.sufficient_share:
                type_leaf = self.LEAF_TYPE
            if y.shape[0] <= self.min_samples_split:
                 type_leaf = self.LEAF_TYPE

            if type_leaf == self.LEAF_TYPE:
                self.__process_leaf_node(y, node)
                continue

            #дальше определяем по каким признакам будем считать
            features = self.get_feature_ids(x.shape[1])
            find_feature = 0
            find_value = None
            find_threshold = None
            find_gs = None
            for feature in features:
                value, threshold = self.__find_threshold( x[:, feature], y)
                if value == None:
                    continue
                if find_value==None or find_value > value:
                    find_feature = feature
                    find_value = value
                    find_threshold = threshold

            if find_value == None:
                #случай ошибки, если не смогли найти разделения
                self.__process_leaf_node(y, node)
                continue


            x_l, x_r, y_l, y_r =self.__div_samples(x, y, find_feature, find_threshold)

            self.__process_non_leaf_node(node, find_feature, find_threshold)

            ##учим левую половину
            lst.append([x_l, y_l, node.createLeft(), depth+1])
            ##учим правую половину
            lst.append([x_r, y_r, node.createRight(), depth+1])


    def __process_leaf_node(self, y, node):
        clas, count = np.unique(y, return_counts=True)
        max_class = clas[np.argmax(count)]
        probability = np.max(count) / y.shape[0]
        
        node.type_node = self.LEAF_TYPE
        node.clazz = max_class 
        node.proba = probability
    
    def __process_non_leaf_node(self, node, feature_id, threshold):
        node.type_node = self.NON_LEAF_TYPE
        node.feature_id = feature_id 
        node.threshold = threshold
        
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, self.tree, 0) 

    def __predict_class(self, x, node):
        if node.type_node == self.__class__.NON_LEAF_TYPE:
            feature_id, threshold = node.feature_id, node.threshold
            if x[feature_id] > threshold:
                return self.__predict_class(x, node.getLeft())
            else:
                return self.__predict_class(x, node.getRight())
        else:
            return node.clazz

    def __predict_probs(self, x, node):
        if node.type_node == self.__class__.NON_LEAF_TYPE:
            feature_id, threshold = node.feature_id, node.threshold
            if x[feature_id] > threshold:
                return self.__predict_probs(x, node.getLeft())
            else:
                return self.__predict_probs(x, node.getRight())
        else:
            return node.proba
        
    def predict(self, X):
        return np.array([self.__predict_class(x, self.tree) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, self.tree) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [85]:
df = pd.read_csv('cs-training.csv', sep=',').dropna()
df.shape

(120269, 11)

In [86]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

C:\app\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\app\anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [87]:
np.unique(y, return_counts=True)[1]

array([111912,   8357], dtype=int64)

In [88]:
111912/(111912+8357)

0.930514097564626

По хорошему - очень не равномерный dataset. И тупое предсказание 0 класса дало бы 0,93

In [89]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_depth=8)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [90]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)
print(my_clf.tree)


0.8451054096221924
[0,5,56.5,None,[0,5,57.5,None,[1,(None,),None,1.0,None,None],[1,(None,),None,0.5,None,None]],[0,1,100.0,None,[0,7,1.0,None,[1,(None,),None,1.0,None,None],[0,9,0.5,None,[1,(None,),None,1.0,None,None],[0,1,101.5,None,[1,(None,),None,1.0,None,None],[1,(None,),None,0.5,None,None]]]],[0,4,243745.0,None,[0,5,16.5,None,[1,(None,),None,1.0,None,None],[0,3,0.023822465,None,[1,(None,),None,1.0,None,None],[0,3,0.017881928499999998,None,[1,(None,),None,1.0,None,None],[1,(None,),None,1.0,None,None]]]],[0,7,30.5,None,[1,(None,),None,1.0,None,None],[0,4,234800.0,None,[1,(None,),None,1.0,None,None],[0,4,226637.0,None,[1,(None,),None,1.0,None,None],[0,7,27.5,None,[1,(None,),None,1.0,None,None],[0,7,25.5,None,[1,(None,),None,1.0,None,None],[1,(None,),None,0.9305337225840258,None,None]]]]]]]]]


In [91]:
t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.9506881237030029


Скорость работы медленней, но это связано с частым вызовом функции argsort. 

## Проверка качества работы

In [54]:
gkf = KFold(n_splits=5, shuffle=True)

In [55]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.9321942296499542
0.9307391702003824
0.9324852415398687
0.9289515257337657
0.9280339250821104


In [56]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.8927829051301239
0.8962750478090962
0.8939053795626507
0.8924087469859483
0.8905749802519436


Качество, довольно близко

# Применить для задачи Titanic 

https://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8  - предварительная подготовка выполняется в соответствии с приведенной статьей

In [57]:
import pandas as pd

In [58]:
df = pd.read_csv('titanic.csv', sep='\t')

In [59]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 12 columns):
PassengerId    156 non-null int64
Survived       156 non-null int64
Pclass         156 non-null int64
Name           156 non-null object
Sex            156 non-null object
Age            126 non-null float64
SibSp          156 non-null int64
Parch          156 non-null int64
Ticket         156 non-null object
Fare           156 non-null float64
Cabin          31 non-null object
Embarked       155 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 14.7+ KB


In [61]:
train_df = df.drop(['PassengerId'], axis=1)


In [62]:
import re
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
data = [train_df]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)
# we can now drop the cabin feature
train_df = train_df.drop(['Cabin'], axis=1)


In [63]:
data = [train_df]

for dataset in data:
    mean = train_df["Age"].mean()
    std = train_df["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_df["Age"].astype(int)
train_df["Age"].isnull().sum()


0

In [64]:
common_value = 'S'
data = [train_df]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(common_value)


In [65]:
train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 11 columns):
Survived    156 non-null int64
Pclass      156 non-null int64
Name        156 non-null object
Sex         156 non-null object
Age         156 non-null int32
SibSp       156 non-null int64
Parch       156 non-null int64
Ticket      156 non-null object
Fare        156 non-null float64
Embarked    156 non-null object
Deck        156 non-null int32
dtypes: float64(1), int32(2), int64(4), object(4)
memory usage: 12.3+ KB


In [66]:
data = [train_df]

for dataset in data:
    dataset['Fare'] = dataset['Fare'].fillna(0)
    dataset['Fare'] = dataset['Fare'].astype(int)


In [67]:
data = [train_df]
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    # extract titles
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)
train_df = train_df.drop(['Name'], axis=1)


In [68]:
genders = {"male": 0, "female": 1}
data = [train_df]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)


In [69]:
train_df = train_df.drop(['Ticket'], axis=1)


In [70]:
ports = {"S": 0, "C": 1, "Q": 2}
data = [train_df]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].map(ports)


In [71]:
data = [train_df]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6


In [72]:
data = [train_df]

for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3
    dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4
    dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)


In [73]:
train_df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Title
0,0,3,0,2,1,0,0,0,8,1
1,1,1,1,5,1,0,3,1,3,3
2,1,3,1,3,0,0,0,0,8,2
3,1,1,1,5,1,0,3,0,3,3
4,0,3,0,5,0,0,1,0,8,1
5,0,3,0,3,0,0,1,2,8,1
6,0,1,0,6,0,0,3,0,5,1
7,0,3,0,0,3,1,2,0,8,4
8,1,3,1,3,0,2,1,0,8,3
9,1,2,1,1,1,0,2,1,8,3


In [74]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]


In [75]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [76]:
for train, test in gkf.split(X_train.values, Y_train.values):
    x_train, y_train = X_train.iloc[train], Y_train.iloc[train]
    x_test, y_test = X_train.iloc[test], Y_train.iloc[test]
    clf.fit(x_train, y_train)
    print(accuracy_score(y_pred=clf.predict(x_test), y_true=y_test))

0.65625
0.7419354838709677
0.7741935483870968
0.7741935483870968
0.7741935483870968


In [94]:
for train, test in gkf.split(X_train.values, Y_train.values):
    x_train, y_train = X_train.iloc[train], Y_train.iloc[train]
    x_test, y_test = X_train.iloc[test], Y_train.iloc[test]
    my_clf.fit(x_train.values, y_train.values)
    print(accuracy_score(y_pred=my_clf.predict(x_test.values), y_true=y_test.values))

0.625
0.7096774193548387
0.6774193548387096
0.7096774193548387
0.8064516129032258


Результаты близки

In [78]:
from sklearn.model_selection import ParameterGrid


In [79]:
from sklearn.model_selection import GridSearchCV

In [80]:
depths = [10,20,30]


In [93]:
for depth in depths:
    my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_depth=depth)
    for train, test in gkf.split(X_train.values, Y_train.values):
        x_train, y_train = X_train.iloc[train], Y_train.iloc[train]
        x_test, y_test = X_train.iloc[test], Y_train.iloc[test]
        my_clf.fit(x_train.values, y_train.values)
        print(accuracy_score(y_pred=my_clf.predict(x_test.values), y_true=y_test.values), depth)

0.6875 10
0.9032258064516129 10
0.5806451612903226 10
0.8387096774193549 10
0.7096774193548387 10
0.71875 20
0.6129032258064516 20
0.6451612903225806 20
0.7741935483870968 20
0.6451612903225806 20
0.75 30
0.7419354838709677 30
0.8064516129032258 30
0.6774193548387096 30
0.6774193548387096 30


Лучше при максимальной глубине 20. Дальше начинается переобучение